In [1]:
%matplotlib inline
import os
import glob

import pandas as pd
import geopandas as gpd

import folium
from folium.plugins import MarkerCluster
from shapely.geometry import Point

In [2]:
datadir = '../data/DHM_data/'

In [3]:
def convert2dd(*args):
    return args[0] + args[1]/60 + args[2]/3600

In [4]:
def convert2ddmet(*args):
    return args[0] + args[1]/60

In [5]:
def create_popup(feat):
    text = '<b>{key}</b>: {val}'.format
    text_list = [text(key=k, val=v) for k, v in feat.to_dict().items()]
    
    return '<p>{}</p>'.format('<br>'.join(text_list))

In [6]:
center = [28.06476680384088, 83.96989711934157]

In [7]:
m = folium.Map(location=center, zoom_start=5, tiles="cartodbpositron")

## Hydro

In [8]:
xlsx = os.path.join(datadir, 'Hydrological Stations List_01222018.xlsx')

In [9]:
rawdf = pd.read_excel(xlsx, sheet_name=1, skip_footer=4)

In [10]:
rawdf.loc[0:60, 'priority'] = 'high'

In [11]:
rawdf.loc[61:, 'priority'] = 'low'

In [12]:
df = rawdf.drop(61).reset_index(drop='index')

In [13]:
df.loc[:, 'lat'] = df.apply(lambda row: convert2dd(*tuple(map(lambda x: int(x), row['Latitude\n(ON)'].split(' ')))), axis=1)

In [14]:
df.loc[:, 'lon'] = df.apply(lambda row: convert2dd(*tuple(map(lambda x: int(x), row['Longitude\n(OE)'].split(' ')))), axis=1)

In [15]:
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

In [16]:
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs={'init': 'epsg:4326'})

In [51]:
gdf.to_csv(os.path.join('../data/', 'hydro_stations.csv'), index=False)

In [17]:
hydro_stations = MarkerCluster(name='Hydrological Stations DHM')

In [18]:
for idx, feature in gdf.iterrows():
    if feature['priority'] == 'high':
        folium.Marker(location=[feature.geometry.y, feature.geometry.x], 
                      icon=folium.Icon(color='red', icon='tint'),
                      popup=create_popup(feature)).add_to(hydro_stations)
    if feature['priority'] == 'low':
        folium.Marker(location=[feature.geometry.y, feature.geometry.x], 
                      icon=folium.Icon(color='blue', icon='tint'),
                      popup=create_popup(feature)).add_to(hydro_stations)

In [19]:
hydro_stations.add_to(m)

## Meteorological

In [20]:
mxlsx = os.path.join(datadir, 'Meteorological Stations List_01222018.xlsx')

In [21]:
mrawdf = pd.read_excel(mxlsx, header=0, skip_footer=5)

In [22]:
fmrawdf = mrawdf[['S. N.', 'DHM S.N.', 'Station name', 'Index No.', 'Basin Office',
       'Type of station', 'District', 'Lat.', 'Log.', 'Ele.', 'Data Type', 
        'From', 'To', 'No. of Years', 'Frequency']]

In [23]:
fmrawdf.loc[0:198, 'priority'] = 'high'

/home/lsetiawan/Miniconda/envs/himat/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/lsetiawan/Miniconda/envs/himat/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [24]:
fmrawdf.loc[201:, 'priority'] = 'low'

/home/lsetiawan/Miniconda/envs/himat/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [25]:
mdf = fmrawdf.drop([0, 199, 200]).reset_index(drop='index')

In [26]:
mdf.loc[:, 'lat'] = mdf.apply(lambda row: convert2ddmet(int(str(int(row['Lat.']))[:-2]), int(str(int(row['Lat.']))[-2:])), axis=1)

In [27]:
mdf.loc[:, 'lon'] = mdf.apply(lambda row: convert2ddmet(int(str(int(row['Log.']))[:-2]), int(str(int(row['Log.']))[-2:])), axis=1)

In [28]:
mgeometry = [Point(xy) for xy in zip(mdf['lon'], mdf['lat'])]

In [29]:
mgdf = gpd.GeoDataFrame(mdf, geometry=mgeometry, crs={'init': 'epsg:4326'})

In [49]:
mgdf.to_csv(os.path.join('../data/', 'met_stations_existing.csv'), index=False)

In [30]:
met_stations = MarkerCluster(name='Meteorological Stations DHM')

In [31]:
for idx, feature in mgdf.iterrows():
    if feature['priority'] == 'high':
        folium.Marker(location=[feature.geometry.y, feature.geometry.x], 
                      icon=folium.Icon(color='red', icon='cloud'),
                      popup=create_popup(feature)).add_to(met_stations)
    if feature['priority'] == 'low':
        folium.Marker(location=[feature.geometry.y, feature.geometry.x], 
                      icon=folium.Icon(color='blue', icon='cloud'),
                      popup=create_popup(feature)).add_to(met_stations)

## Met (Closed)

In [32]:
clsd = pd.read_excel(mxlsx, sheet_name=1, header=2)

In [33]:
clsd.columns

Index(['S.N.', 'Station name', 'Index No.', 'Unnamed: 3', 'Type of station',
       'District', 'Lat.', 'Log.', 'Ele.', 'Ag.Estd', 'NMS Estd.',
       'Unnamed: 11', 'Ag.Clsd.', 'NMS Clsd.', 'Remarks', 'Remark1'],
      dtype='object')

In [34]:
clsddf = clsd[['S.N.', 'Station name', 'Index No.', 'Type of station', 'Lat.', 'Log.', 'Ele.']].drop([0, 81]).reset_index()

In [35]:
clsddf.loc[:, 'lat'] = clsddf.apply(lambda row: convert2ddmet(int(str(int(row['Lat.']))[:-2]), int(str(int(row['Lat.']))[-2:])), axis=1)

In [36]:
clsddf.loc[:, 'lon'] = clsddf.apply(lambda row: convert2ddmet(int(str(int(row['Log.']))[:-2]), int(str(int(row['Log.']))[-2:])), axis=1)

In [37]:
clsdgdf = gpd.GeoDataFrame(clsddf, geometry=[Point(xy) for xy in zip(clsddf['lon'], clsddf['lat'])], crs={'init': 'epsg:4326'})

In [50]:
clsdgdf.to_csv(os.path.join('../data/', 'met_stations_closed.csv'), index=False)

In [38]:
for idx, feature in clsdgdf.iterrows():
    folium.Marker(location=[feature.geometry.y, feature.geometry.x], 
                  icon=folium.Icon(color='gray', icon='cloud'),
                  popup=create_popup(feature)).add_to(met_stations)

In [39]:
met_stations.add_to(m)

## IMD

In [40]:
imddir = '../data/IMD_data/Shapefiles/'

In [41]:
shplist = glob.glob(os.path.join(imddir, '*.shp'))

In [42]:
def create_feat_group(shp):
    fname = os.path.basename(shp)[:-4]
    imdgdf = gpd.GeoDataFrame.from_file(shp)
    imd_stations = MarkerCluster(name=fname)
    icon = 'tint'
    if 'AWS' in fname:
        icon = 'cloud'
    for idx, feature in imdgdf.iterrows():
        folium.Marker(location=[feature.geometry.y, feature.geometry.x], 
                      icon=folium.Icon(color='purple', icon=icon),
                      popup=create_popup(feature)).add_to(imd_stations)
    return imd_stations

In [43]:
shplist

['../data/IMD_data/Shapefiles/RaingaugeIMD.shp',
 '../data/IMD_data/Shapefiles/AWSIMD.shp']

In [44]:
for shp in shplist:
    create_feat_group(shp).add_to(m)

In [45]:
folium.LayerControl().add_to(m)

In [46]:
m.save('../data/HiMATStations.html')